In [33]:
# coding : utf-8

# Projet Python : reconnaissance de mélanomes 

Louise Blart & Jeanne Astier

## Introduction 

Les grains de beauté, ou *nævus mélanocytaire*, sont des tâches présentes sur la peau. La plupart des grains de beauté sont bénins ; mais certains peuvent évoluter en tumeurs malignes, dans 10 cas pour 100 000 environ. Ils s'agit alors de __mélanomes__, une forme de cancer de la peau.

Comme pour toutes les formes de cancer, un diagnostic précoce augmente l'efficacité du traitement : une mélanome détecté tôt peut être traité facilement, avant que la maladie n'évolue vers une forme mortelle. 

### Problématique

Comment la data science peut-elle aider à la reconnaissance des mélanomes ? 

Ce projet a vocation à __aider au diagnostic de mélanomes__ : nous voulons détecter, parmi un ensemble grains de beauté, lesquels sont malins et présentent un danger pour le patient. Pour ce faire, nous construisons sur algorithme de classification d'images distingant les grains de beauté "malins" et "bénins". 

NB : cette problématique a constitué l'objet d'un [challenge sur la plateforme Kaggle](https://www.kaggle.com/c/siim-isic-melanoma-classification), auquel ont participé plus de 3 000 équipes. Dans la mesure où nous sommes débutantes en machine learning, notre objectif n'est absolument pas de rivaliser avec tous ces participants : nous ne saurions prétendre concurrencer les gagnants d'un tel challenge en termes d'efficacité du diagnostic. Notre approche serait pluôt : comment, en partant de zéro en machine learning, peut-on essayer de traiter des images médicales pour y poser un diagnotic ?

### Plan

Notre projet se décompose en trois parties :
1. [Récupération et traitement des données](#1.-Récupération-et-traitement-des-données)
2. [Analyse de la base](#2.-Analyse-de-la-base)
3. [Modélisation et diagnostic](#3.-Modélisation-et-diagnostic)


### Préalable : téléchargement des modules

In [1]:
# Indiquez ici le chemin vers le dossier où se trouvent les modules téléchargés sur Github ( Recuperation_des_donnees , rapport, ...  )
path_files_modules = r'C:\Users\jeann\OneDrive\Documents\scolaire\ENSAE\2A\S1\python\projet\GIT'

In [3]:
# IIIIIIIIIIIIIIIICCCCCCIIIIIIIIIIII
# modules classiques 
import numpy as np
import pandas as pd
import csv
import os 
import random
import sys

# modules créés pour ce projet 
sys.path.insert(0, path_files_modules)
import Recuperation_des_donnees as RD

# pour télécharger les données
import urllib.request
from zipfile import ZipFile

# pour la visualisation des données
import matplotlib.pyplot as plt

# modules de traitement d'images
import cv2  # Utiliser pip install opencv-python et redemarrer le noyau si besoin
import PIL

# modules de traitement du format DICOM
import pydicom as dicom  # Utiliser pip install pydicom et redemarrer le noyau si besoin
import pydicom.data
from pydicom.pixel_data_handlers.util import convert_color_space 

## 1. Récupération et traitement des données

Nous avons choisi d'effectuer ce projet sur le thème de la reconnaissance de mélanomes pour deux raisons : 
- il s'agit d'une forme de reconnaissance d'images médicales relativement accessible (plus accessible que le traitement d'images médicales comme des scanners ou des IRM par exemple) ;
- une large base d'images de grains de beauté est mise à disposition librement par la SIIM (*Society for Imaging Informatics in Medicine*) et l'ISIC (*International Skin Imaging Collaboration*). 

Le premier enjeu, et pas des moindres, est donc la récupération de ces données et leur traitement pour pouvoir ensuite les exploiter. 

### a) téléchargement des données

Les données sont disponibles [sur le site de l'ISIC](https://challenge2020.isic-archive.com/). Il s'agit de photos de grains de beauté, chacune étant accompagnée de métadonnées (âge et sexe du patient, partie du corps concerncée, etc.). Ces données sont disponibles sous plusieurs formats : 
- au format DICOM (*Digital Imaging and Communications in Medecine*) : il s'agit d'un format standard international pour la gestion informatique des données issues de l'imagerie medicale. Chaque fichier .dcm comprend une image et les métadonnées s'y rapportant. 
- aux formats JPG (pour les images) et CSV (pour les métadonnées) : ces formats sont également proposés par l'ISIC car plus facilement utilisables par le grand public. 

Nous avons fait le choix d'__utiliser des données au format DICOM__ et de les retraiter nous-mêmes pour les adapter à nos besoins. Nous souhaitons en effet inscrie ce projet dans une perspective médicale, en utilisant donc les données telles qu'elles se présentent en imagerie médicale.  


Création d'un dossier Projet_Melanomes comprenant : 
<li> ISIC_2020_Training_Dicom: base de données complete comprenant 33 106 images de grains de beauté avec leurs métadonnées associées</li>
<li> Diagnostic : fichier CSV complémentaire à la base ISIC_2020_Training_Dicom nous indiquant le diagnostic de chaque grain de beauté (bénin ou malin) </li>

In [4]:
RD.Premiere_fonction ()

 Répondez par 1 pour 'Oui' et 0 pour 'Non' à ce questionnaire 
Avez vous déja lancé ce programme ? (le dossier "Projet_Melanomes" contenant le fichier Base_complete (ISIC_2020_Training_Dicom dezippé) et le fichier Diagnostic est-il deja créé ?) [1 :"oui", 0: "non"] 1
Insérez le chemin du document 'Projet_Melanomes' (exemple : C:/Users/louis/OneDrive/Bureau/Projet_Melanomes) : D:\Projet_Melanomes


A la fin de cette fonction, vous devez avoir : 
- Le dossier **" Projet_Melanomes"** contenant : 
    - le dossier **"ISIC_2020_Training_Dicom"** ( fichier zip téléchargé depuis le site https://challenge2020.isic-archive.com/ )
    - le dossier **"Base_complete"** ( extraction du fichier ISIC_2020_Training_Dicom) contenant : 
        - le dossier "train" contenant : 
            - les images au format dicom 
    - le dossier **"Dicom_Sample_Test"**, pour le moment vide, il contiendra les images au format **dicom** de notre base de **test** 
    - le dossier **"Dicom_Sample_Train"**, pour le moment vide, il contiendra les images au format **dicom** de notre base de **train**
    - le dossier **"JPG_Sample_Test"**, pour le moment vide, il contiendra les images au format **jpeg** de notre base de **test**
    - le dossier **"JPG_Sample_Test_Resize"**, pour le moment vide, il contiendra les images **réduites** et au format **jpeg** de notre base **test**
    - le dossier **"JPG_Sample_Train"**, pour le moment vide il contiendra des images au format **jpeg** de notre base **train** 
    -  le dossier **"JPG_Sample_Train_Resize"**, pour le moment vide, il contiendra les images **réduites** au format **jpeg** de notre base **train** 
    - le fichier .csv **"Diagnostic"** téléchargé à partir du site https://challenge2020.isic-archive.com/, rapportant le diagnostic des patients (mélin ou malin) 
    
    
Au cours de ce rapport, nous allons ajouter dans le dossier  **" Projet_Melanomes"**  :
- le fichier **"Base_complete"** au format .csv;  dataframe contenant l'ensemble de nos métadonnées 
- le fichier **"Sample"** au format .csv; dataframe contenant l'ensemble des métadonnées de l'échantillon selectionné

In [5]:
# Chemin vers le dossier Projet_Melanomes créé à partir de la fonction ci-dessus
Path_Projet_Melanomes= RD.Path_Projet_Melanomes

### b) traitement des données

Maintenant que nous avons téléchargé toutes les données, nous pouvons commencer à les explorer - en particulier regarder comment se présente le format DICOM (format des images). 

In [ ]:
# on choisit un fichier DICOM au hasard pour regarder sa structure 
path_dicom = Path_Projet_Melanomes+'/Base_complete/train'
files = os.listdir(path_dicom)
i = random.randint(0, len(files))
file = files[i]

In [ ]:
# affichage des métadonnées associées à l'image
filename = pydicom.data.data_manager.get_files(path_dicom, file)[0]
ds = pydicom.dcmread(filename)
print(ds) 

On constate que les métadonnées sont très nombreuses ; mais seules quelques-unes nous intéressent : 

In [ ]:
print("image :", file)
print("nom du patient :", ds.PatientName)
print("âge du patient :", ds.PatientAge)
print("sexe du patient :", ds.PatientSex)
print("prtie du corps : ", ds.BodyPartExamined)

On peut enfin afficher l'image en tant que telle : 

In [ ]:
plt.imshow(ds.pixel_array) 

On constate que les couleurs de l'image n'ont pas l'air naturelles : il faut changer leur format de couleur pour avoir une aperçu des couleurs "naturelles" (format RGB). 
        

In [ ]:
initial_color = ds.PhotometricInterpretation
convert = convert_color_space(ds.pixel_array, initial_color, 'RGB')
plt.imshow(convert)

Comme nous venons de le voir, le format DICOM peut être manipulé sur python à l'aide du module "pydicom". Cependant, ce format ne nous apparaît pas comme le plus facilement manipulable : nous préférons donc convertir ces données dans un format avec lequel nous sommes plus à l'aise. 

Deux étapes dans le traitement des données : 
- l'extraction des métadonnées de chaque fichier DICOM, pour les insérer dans un dataframe. 
- l'extraction des images de chaque fichier DICOM, pour les enregistrer au format JPG. 

Pour ce faire, nous avons créé une classe Dataframe dans le module Recuperation_des_donnees nous permettant d'extraire les métadonnées et de changer le format des images.

In [ ]:
help(RD.Dataframe)

In [6]:
w=RD.Dataframe()
# Mise à jour de notre classe en fonction de nos chemins d'accès 
w.path_base_complete = Path_Projet_Melanomes+'/Base_complete/train'
w.path_Diagnostic= Path_Projet_Melanomes+'/Diagnostic.csv'

Nous traitons maintenant l'ensemble des fichiers DICOM téléchargés pour les insérer dans le dataframe des métadonnées. 

( __/!\ attention__ : là encore, la cellule suivant peut prendre plusieurs heures à tourner. Le programe doit en effet passer en revue quelques 33 106 fichiers pour en extraire des données, ce qui peut être un peu long).

Afin de limiter le temps de chargement, vous pouvez trouver ce fichier directement en format .csv sur le drive https://drive.google.com/drive/folders/1ByHZayDJD6OiB7g9D3hHsUMFWFZmeBy3?usp=sharing. Merci de l'enregistrer sous le nom **"Base_complete"** (format .csv) dans le dossier **"Projet_Melanomes"**. Les deux prochaines cellules ne sont pas à faire tourner dans le cas d'un téléchargement sur le Drive.

In [ ]:
df = w.from_DICOM_to_DF()
df.head(100)

Comme cette étape de création et de remplissage du datagrame complet est très très longue, nous préférons éviter de devoir la relancer à chaque nouvelle ouverture du projet ; aussi enregistrons-nous ce dataframe au format csv, pour pouvoir ensuite l'ouvrir plus facilement et rapidement. 

In [ ]:
df.to_csv(Path_Projet_Melanomes+'/Base_complete.csv', index = False)

## 2. Analyse de la base

Nous allons maintenant analyser la base ainsi construite, et la représenter sous forme de graphiques pour se familiariser avec les données. 

Au préalable, nous téléchargeons les modules nécessaires :


In [7]:
#visualisation
import seaborn as sns

#Tests statistiques 
import scipy.stats as stats

Nous rehchargons la base à partir du CSV précédemment enregistré, pour éviter de devoir relancer le téléchargement de la base à chaque fois. 

In [8]:
df=pd.read_csv(Path_Projet_Melanomes+'/Base_complete.csv')
df.head()

,Unnamed: 0,image_id,patient_age,patient_sex,body_part,image_name,target,patient_id
0,0,ISIC_0015719,40,F,UPPER EXTREMITY,ISIC_0015719,0,IP_3075186
1,1,ISIC_0052212,50,F,LOWER EXTREMITY,ISIC_0052212,0,IP_2842074
2,2,ISIC_0068279,45,F,HEAD/NECK,ISIC_0068279,0,IP_6890425
3,3,ISIC_0074268,55,F,UPPER EXTREMITY,ISIC_0074268,0,IP_8723313
4,4,ISIC_0074311,40,F,LOWER EXTREMITY,ISIC_0074311,0,IP_2950485


In [ ]:
df.shape

On remarque que cette base est volumineuse : 33 126 lignes, c'est-à-dire 33 126 images de mélanomes (le nombre de patient peut être moindre : un même patient peut avoir plusieurs images de ménalomes). 

Par manque de puissance de calculs, nous ne sommes pas en mesure de traiter une telle base de données. C'est pourquoi nous souhaitons l'échantillonner avant de créer un algorithme de machin learning permettant d'identifier les mélanomes bénins et malins.
Comment selectionner notre échantillon ? 


In [ ]:
df.isnull().sum()

Cette commande permet d'identifier le nombre de valeurs manques. Il n'y en a ici que très peu : seulement 65 patients sur 33 126 ont omis de renseigner leur genre (valeur 'nan') et 49 patients ont renseigné 'X'. 
Les valeurs non-exploitables de la variable 'Patient_sex' représentent moins de 0,3% de notre échantillon, nous ne ferons donc pas d'étude sur les non réponses, et retirons ces patients de notre base de données. 


In [ ]:
indexNames = df[ df['patient_sex']=='X'].index
df.drop(indexNames , inplace=True)
df.dropna(inplace=True)
# Enregistrement sur notre fichier.csv
df.to_csv(Path_Projet_Melanomes+'/Base_complete.csv', index = False)

### 1 - Analyses univariées

#### a) L'âge

In [ ]:
df['patient_age'].hist( facecolor='b', alpha=0.5)
plt.title('Age des patients')
plt.show()
print('Moyenne :', df['patient_age'].mean(), '\n', 'Ecart type:',  df['patient_age'].std())

#### b) Le sexe

In [ ]:
df.groupby('patient_sex')['patient_id'].nunique()

In [ ]:
df["patient_sex"].value_counts().plot(kind='pie' , autopct='%1.1f%%')
plt.xlabel(' Parité Hommes/Femmes', fontsize=15)

La parité Homme/Femme est respectée. 

#### c) La partie du corps

In [ ]:
df['body_part'].value_counts()

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x=df['body_part'].unique(), y=df['body_part'].value_counts(), palette="Reds_r")
plt.xlabel('\nParties du corps', fontsize=15, color='#c0392b')
plt.ylabel("Nombre de grains de beauté\n", fontsize=15, color='#c0392b')
plt.title("Localisation des grains de beauté\n", fontsize=18, color='#e74c3c')
plt.xticks(rotation= 45)
plt.tight_layout()

#### d) Melanomes bénins et malins

In [ ]:
plt.pie([sum(df['target']==1),sum(df['target']==0)], labels = ['Malins','Benins'],colors = ['lightcoral','lightskyblue'],autopct='%1.1f%%')
plt.title("Taux de melanomes benins et malins dans l'échantillon")
plt.show()

#### e) Patients

In [ ]:
print("nombre d'images :", df["image_id"].nunique())
print("nombre de patients :", df["patient_id"].nunique())

Il y a bien moins de patients que d'images (33 012 images pour 2 051 patients) : plusieurs images peuvent donc appartenir à un même patient (environ 16 images par patient en moyenne)

In [ ]:
df1 = df.groupby("patient_id").sum()["target"]
df2 = df.groupby("patient_id").count()["image_name"]

pd.concat([df1, df2], axis=1, sort=False).sort_values(by = "image_name", ascending = False).head(150)

pd.concat([df1, df2], axis=1, sort=False).sort_values(by = "target", ascending = False).head(150)

pd.concat([df1, df2], axis=1, sort=False).sort_values(by = "target", ascending = False).head(20).plot(kind = "bar", figsize = (25,5))
plt.title("nombre de mélanomes bénins et d'images pour les 20 patients ayant le plus de mélanomes bénins", fontsize = 25)

Il y a jusqu'à 115 images par patient, et jusqu'à 8 mélanomes par patient au sein de la base de donnée. Mais les patients qui ont le plus d'images ne sont pas nécessairement ceux qui ont le plus de mélanomes.

In [ ]:
df1 = df.groupby("patient_id").agg({"target" : "sum"})
print("nombre moyen de mélanomes par patient :", np.average(df1["target"]))

Le nombre moyen de mélanomes par patient est surprenamment élevé : 0,28. Nous attentions un chiffre beaucoup plus bas du fait de la faible proportion du mélanomes parmi les images de la base (moins de 2%). Cela est en fait dû au faible nombre de patients au regard du nombre d'images. Si les ménalomes sont très dilués parmi les images, ils le sont beaucoup moins parmi les patients. 

In [ ]:
print ("nombre de patients n'ayant aucun mélanomes :", len(df1[df1["target"] == 0]))
print ("nombre de patients ayant au moins un mélanome :", len(df1[df1["target"] > 0]))
print("proportion de patients ayant au moins un mélanome :", len(df1[df1["target"] > 0]) / df["patient_id"].nunique() *100, "%")

Plus de 20% des patients de la base ont un mélanome : cette proportion est très très largement supérieure à la proportion de personnes touchées par un mélanome au sein de la population (le taux d'incidence des mélanomes étant d'environ 10 pour 100 000 en France). Cette différence s'explique par le fait que nous disposons d'un échantillon de données médicales sur-représentant volontairement les cas malins, pour apprendre à les diagnostiquer. 

In [ ]:
print("nombre moyen de mélanomes parmi les patients en ayant au moins un :", np.average(df1[df1["target"] > 0]["target"]))
print("nombre de patients ayant plusieurs mélanomes :", len(df1[df1["target"] > 1]))
print("proportion de patients ayant plusieurs mélanomes parmi les patients malades :", len(df1[df1["target"] > 1]) / len(df1[df1["target"] > 0])*100, "%")

Parmi les patients malades (i.e. ayant au moins un mélanome), plus d'un quart présente plusieurs mélanomes (jusqu'à 8, comme vu plus haut). Les mélanomes, en faible nombre dans la base au regard du nombre d'image, sont donc en fait concentrés sur une relativement petite part des patients, la majorité n'ayant que des grains de beauté bénins.

### 2 - Analyses bivariées

#### a) Les mélanomes et l'âge

In [ ]:
plt.figure(figsize=(15,8))
ax = sns.kdeplot(df["patient_age"][df.target == 1], color="darkturquoise", shade=True)
sns.kdeplot(df["patient_age"][df.target == 0], color="lightcoral", shade=True)
plt.legend(['Melanome malin', 'Melanome bénin'])
plt.title("Fonctions de densité : répartition de l'age pour les mélanomes malins et bénins")
ax.set(xlabel='Age')
plt.xlim(-10,90)
plt.show()

La distribution des mélanomes malins est décalée sur la droite par rapport à celle des bénins. Cela signifie que les personnes agées sont plus touchées par les mélanomes malins que les personnes jeunes. Les articles scientifiques confirment cette hypothèse. 


#### b) Melanomes et sexe

In [ ]:
(df.groupby('patient_sex')['target'].sum() / df.groupby('patient_sex')['patient_id'].count() *100)

In [ ]:
sns.barplot(x='patient_sex', y='target', data=df, color="aquamarine")
plt.show()

D'après cet échantillon les hommes sont plus touchés par les mélanomes malins que les femmes. 
1,37% des femmes de l'échantillon sont porteuses d'un mélanome bénins contre 2,13% des hommes.
Les articles scientifiques ne confirment pas cette hypothèse. 


In [ ]:
print(stats.ttest_ind(df["target"][df.patient_sex == 'F'],df["target"][df.patient_sex == 'M']))

D'après les résultats du T-test, les populations féminines et masculines sont significativement différentes en ce qui concerne les mélanomes (p-value < 0,001)

#### c) Mélanomes et partie du corps

In [ ]:
(df.groupby('body_part')['target'].sum() / df.groupby('body_part')['patient_id'].count() *100)

In [ ]:
sns.barplot(x=df['target'], y=df['body_part'],palette='Blues_d', orient='h',  order=["HEAD/NECK", "ORAL/GENITAL","UPPER EXTREMITY","SKIN", "TORSO", "LOWER EXTREMITY", "PALMS/SOLES" ])

Le taux de mélanomes malins est plus élevé sur les parties du corps : 
- tête et cou
- orales et génitales 
- les membres supérieurs
      
Selon les articles scientifiques, les zones les plus exposées aux mélanomes malins sont les parties les plus exposées au soleil.
Il est donc raisonnable d'identifier la tête, le cou et les membres supérieurs dans les parties du corps les plus touchées par les mélanomes malins. 
Dans ce sens, cela est étonnant d'identifier les zones orales et génitales comme zones à risque. Cependant, dans notre échantillon, la catégorie "Oral/Genital" est la moins représentée des parties du corps; il n'y a que 124 images. Ce qui est très faible par rapport à notre base totale. L'échantillon est probablement peu représentatif de la population total pour cette partie du corps.


In [ ]:
print(stats.kruskal(df["target"][df.body_part == 'HEAD/NECK'],df["target"][df.body_part == 'ORAL/GENITAL'],df["target"][df.body_part == 'SKIN'],df["target"][df.body_part == "UPPER EXTREMITY"],df["target"][df.body_part == "TORSO"],df["target"][df.body_part == "PALMS/SOLES"],df["target"][df.body_part == "LOWER EXTREMITY"]))

D'après le test de Kruskal Wallis les populations de mélanomes sont différentes en fonction des parties du corps. 

### 3 - Régression linéaire multiple

La régression linéaire multiple permet de créer des modéles permetant d'expliquer une variable Y à partir des variables explicatives X. Nous allons essayer d'expliquer la variable "targer" à partir de l'age du patient, son sexe et la partie du corps sur laquelle se trouve le grain de beauté.

In [ ]:
training=pd.get_dummies(df, columns=["patient_sex","body_part"])

# On retire la variable patient_sex_F, qui est parfaitement colinéaire à patient_sex_M
training.drop('patient_sex_F', axis=1, inplace=True) 
final_train = training
final_train.head()

In [ ]:
import statsmodels.api as sm
from sklearn import linear_model

In [ ]:
X = final_train[["patient_age", "patient_sex_M", "body_part_HEAD/NECK", "body_part_LOWER EXTREMITY", 
              "body_part_ORAL/GENITAL","body_part_PALMS/SOLES","body_part_SKIN", "body_part_TORSO", "body_part_UPPER EXTREMITY"]]
X = sm.add_constant(X) # une autre façons d'ajouter une constante
y = final_train["target"]

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

In [ ]:
Selected_features = ["patient_age", "patient_sex_M", "body_part_HEAD/NECK", "body_part_LOWER EXTREMITY", 
              "body_part_ORAL/GENITAL","body_part_PALMS/SOLES","body_part_SKIN", "body_part_TORSO", "body_part_UPPER EXTREMITY"]
X = final_train[Selected_features]

plt.subplots(figsize=(10, 7))
sns.heatmap(X.corr(), annot=True, cmap="RdYlGn")
plt.show()

Notre modéle n'est pas convainquant, les coefficients sont très faibles, les variables age, parties du corps et sexe expliquent à hauteur de 0,9% les mélanomes malin (R²=0,009).
Il n'est pas possible d'expliquer convenablement les mélanomes malins à partir de ces variables explicatives.
Il est donc primordial d'analyser les images de mélanomes afin de fournir un modèle prédictif acceptable. 


### 4 - Regression logistique

La régression logistique est une technique prédictive. Elle vise à construire un modèle permettant de prédire / expliquer les valeurs prises par une variable cible qualitative (le plus souvent binaire, on parle alors de régression logistique binaire) à partir d’un ensemble de variables explicatives quantitatives ou qualitatives (un codage est nécessaire dans ce cas).
Nous voulons ici expliquer la variable 'target' (binaire) en fonction des variables 'patient_age'(quantitative), 'patient_sex'(qualitative), 'body_part'(qualitative). 


In [ ]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [ ]:
# On divise la base en deux : 80% dans la base de training et 20% dans la base test. 
X_app,X_test,y_app,y_test = model_selection.train_test_split(X,y,test_size = 6500,random_state=0) # 

In [ ]:
lr = LogisticRegression(solver="liblinear")
# Construction du modéle predictif :  
modele = lr.fit(X_app,y_app)

#Prediction sur l'échantillon test
y_pred = modele.predict(X_test)

In [ ]:
#matrice de confusion : confrontation entre les Y observés sur l’échantillon test et la prédiction

cm = metrics.confusion_matrix(y_test, y_pred)

ax = plt.axes()

sns.heatmap(cm, annot=True, 
           annot_kws={"size": 10}, 
           xticklabels=('Prédiction : Bénin', 'Prédiction : Malin'), 
           yticklabels= ('Benin', 'Malin'), ax = ax)
ax.set_title('Confusion matrix')
plt.show()

In [ ]:
acc = metrics.accuracy_score(y_test,y_pred)
print('Le taux de succés est de ',acc *100,'%')

On pourrait croire que ce modéle de classification est efficace ( il a un taux de succés de 98%). Cependant, cela est dû à la proportion de bénins et de malins de notre base. En effet, le taux de mélanomes malins est très faible ( moins de 2%), notre modéle, en prédisant toujours bénin, arrive donc à limiter ses erreurs. Ce modéle n'est pas satisfaisant pour différencier les mélanomes malins des mélanomes bénins. Il apparait primordial d'utiliser les images afin de rendre une prédiction acceptable.

## 3. Modélisation et diagnostic


import des packages 


In [9]:
#IIIIIIIIIIIIIIIIIIIIIICCCCCCCCCCCCCCCCCCCCCCIIIIIIIIIIIIIIIIIIIIIII
# Module créé pour ce projet 
import Echantillonnage

# package machine learning
import sklearn 

#package TensorFlow (pour le CNN)
import tensorflow as tf    

import shutil 
import pydicom
import seaborn as sns
from tqdm import tqdm
import gc
import sklearn.metrics
import matplotlib.pyplot as plt

### 1 - Création des bases train et test

On veut constituer une base d'entraînement (train) et une base de test à partir de l'ensemble des images (plus de 33 000). 
Cette base de donnée est représentée par le Dataframe "df" importé précedemment. 

In [ ]:
df=pd.read_csv(Path_Projet_Melanomes+'/Base_complete.csv')
df.head()

Calcul du taux de mélanomes malins au sein de cette base :

In [ ]:
t = sum(df['target']==1) / len(df.index)
print("taux de malignité des mélanomes :", t*100, "%")

Comme nous l'avions déjà vu, le taux de malignité des mélanomes est très faible (1,76%).

Aussi, dans un premier temps, pour faciliter la construction de premiers modèles, nous construisons une base réduite avec un taux de malignité supérieur. Nous réduisons ainsi le nombre total d'images, ce qui nous permettra aussi de construire des modèles plus économes en mémoire vive pour l'instant. C'est à partir de cette base réduite que nous constitueront les bases test et train. 

#### a) échantillonnage 

Pour ce faire, on commence par définir une fonction d'échantillonnage simple dans notre module Echantillonnage : 

In [ ]:
help(Echantillonnage.simple_sampling)

Dans un premier temps, nous choisissons d'extraire un échantillon de 1 000 images dont 50% représentent des grains de beauté malins (mélanomes) ; ces paramètres pourront être amenés à varier. 

In [ ]:
np.random.seed(10)
size = 1000
malignancy_rate = 0.5
df_sample = Echantillonnage.simple_sampling(df, size, malignancy_rate)
df_sample.head()

On sauvegarde l'échantillon ainsi construit dans un CSV (au cas où)

In [10]:
# IIIIIIIIIIIIIIIIIIICCCCCCCCCCCCCCCCCCIIIIIIIIIIIIIIIIIIIII
df_sample.to_csv(Path_Projet_Melanomes+'/Sample.csv')

NameError: name 'df_sample' is not defined

#### b) train / test split

Pour diviser cet échantillon en train et test, on s'appuie sur le package scikit-learn. 

Pour l'instant, on choisit arbitrairement d'allouer 20% de notre échantillon au test et 80% au test - proportions que l'on pourra changer par la suite. 

In [15]:
# IIIIIIIIIIIIIIIICCCCCCCCCCCCCCCCCIIIIIIIII
from sklearn.model_selection import train_test_split

X = df_sample["image_id"]
Y = df_sample["target"]

In [16]:
# IIIIIIIIIIIIIIIIIIIICCCCCCCCCCCCCCCCCCIIIIIIIIIII
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)

### 2- Définition des classes et import des données

définition des classes (deux seulement : bénin et malin)

In [17]:
# IIIIIIIIIIIIIIICCCCCCCCCCCCCIIIIIIIIIIIIIIIIIII
class_names = ['benin', 'malin']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)

In [18]:
# IIIIIICCCCCIIIIIIIIIIIIIIIIIIIIIII
class_names_label

{'benin': 0, 'malin': 1}

In [19]:
#IIIIIIIIIIICCCCCCCCCIIIIIIIIIIIIIIIIII
image_size = 150
# peut être modifié en adaptant les paramètres du CNN en conséquences 

In [11]:
df_sample=pd.read_csv(Path_Projet_Melanomes+'/sample.csv')
df_sample.head()

,Unnamed: 0,Unnamed: 0.1,image_id,patient_age,patient_sex,body_part,image_name,target,patient_id
0,89,90,ISIC_0149568,55,F,UPPER EXTREMITY,ISIC_0149568,1,IP_0962375
1,233,234,ISIC_0188432,50,F,UPPER EXTREMITY,ISIC_0188432,1,IP_0135517
2,312,313,ISIC_0207268,55,M,TORSO,ISIC_0207268,1,IP_7735373
3,396,398,ISIC_0232101,60,M,TORSO,ISIC_0232101,1,IP_8349964
4,472,474,ISIC_0250839,75,M,HEAD/NECK,ISIC_0250839,1,IP_6234053


In [ ]:
#création d'un dossier avec les images DICOM de la base train
path_dicom_sample_train = Path_Projet_Melanomes+'/Dicom_Sample_Train'  

for file in X_train :
    shutil.copy(w.path_base_complete + '/' + file +'.dcm', path_dicom_sample_train + '/' + file +'.dcm')

In [ ]:
#création d'un dossier avec les images DICOM de la base test
path_dicom_sample_test =  Path_Projet_Melanomes+'/Dicom_Sample_Test' 

for file in X_test :
    shutil.copy(w.path_base_complete + '/' + file +'.dcm', path_dicom_sample_test + '/' + file +'.dcm')

In [ ]:
#conversion des images train en jpg - attention cette fonction est longue à exécuter (plusieurs dizaines de minutes)
w.path_base_complete = path_dicom_sample_train
w.path_jpg_RGB=Path_Projet_Melanomes+'/JPG_Sample_Train' 
w.convert_to_JPG_RGB()

In [ ]:
#conversion des images test en jpg - attention cette fonction est longue à exécuter (plusieurs dizaines de minutes)
w.path_base_complete = path_dicom_sample_test
w.path_jpg_RGB=Path_Projet_Melanomes+'/JPG_Sample_Test' 
w.convert_to_JPG_RGB()

On redimensionne les images du dossier *Sample_Test* et *Sample_Train*, et on enregistre les images ainsi redimensionnées dans les dossiers *Sample_Test_Resize* et *Sample_Train_Resize* :


In [ ]:
# Pour le dossier Train : (Quelques minutes)
w.path_jpg_RGB = Path_Projet_Melanomes+'/JPG_Sample_Train'
w.path_jpg_Resize = Path_Projet_Melanomes+'/JPG_Sample_Train_Resize'
w.redimensionner((image_size, image_size))

In [ ]:
# Pour le dossier Test : (Quelques minutes)
w.path_jpg_RGB = Path_Projet_Melanomes+'/JPG_Sample_Test'
w.path_jpg_Resize = Path_Projet_Melanomes+'/JPG_Sample_Test_Resize'
w.redimensionner((image_size, image_size))

Il faut ensuite charger les données pour pourvoir les insérer dans le modèle (attention, cette cellule prend un peu de temps à être exécutée, de l'ordre de quelques minutes) :

In [22]:
# IIIIIIIIIIIICCCCCCCCCCCCCCCCCCCCCCIIIIIIIIIIIIIIIIII
path_jpg_train = Path_Projet_Melanomes+'/JPG_Sample_Train_Resize' 
path_jpg_test = Path_Projet_Melanomes+'/JPG_Sample_Test_Resize' 
    
meta_data = df_sample
meta_data = meta_data.set_index("image_id")
datasets = [path_jpg_train, path_jpg_test]

output = []

#itération sur chaque dataset (train puis test)
for dataset in datasets :

    files = os.listdir(dataset)
    images = np.zeros((len(files), image_size, image_size, 3), dtype=np.float32)
    labels = np.zeros(len(files))

    # itération sur chaque image du dataset
    for i, file in tqdm(enumerate(files)):
        
        # obtention du le chemin de l'image
        img_path = os.path.join(dataset, file)

        # obtention du label de l'image
        label = meta_data.loc[file[0:12]].target

        # ouverture de l'image et retraitement de l'image (changement de couleurs et réduction de taille)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (image_size, image_size))

        # ajout de l'image et de son label à la liste de résultats
        images[i, :] = (np.array(image) / 255).astype(np.float32)
        labels[i] = label

        gc.collect()

    output.append((images, labels))

800it [00:45, 17.74it/s]
200it [00:11, 17.36it/s]


on renomme les données :

In [23]:
# IIIIIIIIIIIIIICCCCCCCCCIIIIIIIIII
(train_images, train_labels),(test_images, test_labels) = output

on mélanges les données de la base test pour ne plus avoir les malins en haut et les bénins en bas :

In [24]:
# IIIIIIIIIIIIIIIIIICCCCCCCCCCCCCCCCCCCCCCIIIIIIIIIIIIIIIIIII
train_images, train_labels = sklearn.utils.shuffle(train_images, train_labels, random_state=25)

### 3 - Visualisation des données


Exploration de l'échantillon de données

In [ ]:
n_train = train_labels.shape[0]
n_test = test_labels.shape[0]

print ("nombre d'images dans la base train :", n_train)
print ("nombre d'images dans la base test :", n_test)
print ("taille de chaque image :", image_size)

Visualisation d'exemples d'images de la base, pour en avoir un aperçu : 

In [ ]:
def display_examples(class_names, images, labels):
   
    fig = plt.figure(figsize=(15,10))
    fig.suptitle("Exemples d'images de la base train", fontsize=20)
    for i in range(15):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(images[i])
        plt.xlabel(class_names[labels[i].astype(np.integer)])
    plt.show()

display_examples(class_names, train_images, train_labels)

### 4 - Modèle de CNN

Un réseau de neurones convolutifs est un système composé de neurones, généralement répartis en plusieurs couches connectées entre elles. Nous l'utilison ici pour résoudre un problème de classification. Le réseau calcule à partir de l'entrée une probabilité pour chaque classe. 
Chaque couche reçoit en entrée des données et les renvoie transformée. Pour cela, elle calcule une combinaison linéaire puis applique éventuellement une fonction non-linéaire, appelée fonction d'activation. Les coefficients de la combinaison linéaire définissent les paramètres (ou poids) de la couche.
Un réseau de neurones est construit en empilant les couches : la sortie d'une couche correspond à l'entrée de la suivante.
La dernière couche calcule les probabilités finales en utilisant pour fonction d'activation la fonction logistique (pour une classification binaire).
Une fonction de perte (loss function) est associée à la couche finale pour calculer l'erreur de classification.   
N'ayant pas encore fait de cours de Machine Learning à l'ENSAE, nous nous sommes basées sur les ressources suivantes :    
<li> cours openclassrooms Classez et segmentez des données visuelles : https://openclassrooms.com/fr/courses/4470531-classez-et-segmentez-des-donnees-visuelles</li>
<li> https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/</li>

Les CNN réalisent eux-mêmes le travail fastidieux d'extraction et de description de features, cela constitue une des forces des réseaux de neurones convolutifs : plus besoin d'implémenter un algorithme d'extraction "à la main", comme SIFT ou Harris-Stephens. 
Comme nous l'avons vu précédemment, un reseau de neurones est constitué de plusieurs couches. Il existe quatre types de couches pour un réseau de neurones convolutif : 
<p>
<li>la couche de convolution,</li> 
<li> la couche de pooling, </li> 
<li> la couche de correction ReLU, </li> 
<li> la couche fully-connected.</li> 
</p>

#### 1- Premier modèle simple :

##### a) construction du modèle

In [ ]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (5, 5), activation = 'relu', input_shape = (150, 150, 3)), 
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),  
    
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    
    tf.keras.layers.Conv2D(128, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])


In [ ]:
model2.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

##### b) Explications du modèle : 

**La couche de convolution** constitue toujours au moins la première couche d'un CNN.Son but est de repérer la présence d'un ensemble de features dans les images reçues en entrée. La couche de convolution reçoit donc en entrée plusieurs images, et calcule la convolution de chacune d'entre elles avec chaque filtre. Les filtres correspondent exactement aux features que l'on souhaite retrouver dans les images. 


*tensorflow.keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', data_format=None, dilation_rate=(1, 1), activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)*

   
<p>
<li> Conv2D : pour les images, Conv1D: dimension 1, Conv3D: pour les volumes </li> 
<li> Filters (32) : Nombre de filtres a apprendre : commencer par un nombre de filtre assez faible (32) et augmenter dans les couches suivantes dans l'idéal des multiples de 2, on peut apprendre dans les premières couches 32, 64, 128 filtres puis aller jusqu'à 256, 512, 1024 dans les couches plus pronfondes. </li> 
<li> kernel_size (5,5) : dimension du kernel, si on a des images jusqu'à 128x128 on va utiliser 3x3. Au dessus on utilisera 5x5 ou 7x7. Il faudra dans les couches suivantes se reduire à du 3x3. </li> 
<li> strides : pas avec lequel on va analyser l'image pixel par pixel. La valeur par défaut est (1, 1); cependant, on peut parfois l'augmenter à (2, 2) pour aider à réduire la taille du volume de sortie.  </li> 
<li> Padding : Il peut prendre deux paramétres "valide" ou "same". Avec le paramètre valide, le volume d'entrée n'est pas rempli de zéro et les dimensions spatiales peuvent être réduites via l'application naturelle de la convolution. On preferera reduire les dimensions spaciales avec le max pooling ou la strided convolution. On notera padding = 'same' pour la majorité de nos couches. </li> 
<li> data_format: Height, Width, Depth </li>
<li> dilation_rate : On utilise ce parametre de dilatation quand on travaille avec des images de plus haute résolution (où les détails sont importants) ou quand on construit un réseau avec moins de paramètres. </li>
<li> activation : Nom de la fonction d'activation que l'on souhaite appliquer après avoir effectué la convolution. Elle constitue la couche d'activation. On utilisera ici la fonction 'relu' qui est très utilisée dans les CNN. Elle permet un apprentissage plus rapide de notre modéle.  </li>
<li> use_bias : Contrôle si un vecteur de biais est ajouté à la couche de convolution. Recommandé de garder le biais sauf dans des cas particuliers.</li>
<li> kernel_initializer :contrôle la méthode d'initialisation utilisée pour initialiser toutes les valeurs avant d'entraîner réellement le réseau. Recommandé de ne pas y toucher sauf si réseau très profond</li>
<li> bias_initializer : contrôle la façon dont le vecteur de biais est initialisé avant le début de l'entraînement. Recommandé de ne pas y toucher</li>
<li> kernel_regularizer, bias_regularizer, activity_regularizer : controle le type de regularisation. Recommandé de ne pas changer les valeurs par défaut.  </li>
<li> kernel_constraint,  bias_constraint : Ces paramètres permettent d'imposer des contraintes sur la couche Conv2D (ex : non-négativité, la normalisation d'unité et la normalisation min-max ... ). Il est recommandé de laisser les valeur par defaut.</li>
</p>

**La couche de pooling :** Cette couche est souvent placé entre deux couches de convolution : elle reçoit en entrée plusieurs feature maps, et applique à chacune d'entre elles l'opération de pooling. L'opération de pooling consiste à réduire la taille des images, tout en préservant leurs caractéristiques importantes. La couche de pooling permet de réduire le nombre de paramètres et de calculs dans le réseau. On améliore ainsi l'efficacité du réseau et on évite le sur-apprentissage.


*tf.keras.layers.MaxPooling2D ( pool_size=(2, 2), strides=None, padding="valid", data_format=None)*

<p>
<li> pool_size : facteur de réduction d'échelle (vertical, horizontal). (2, 2) réduira de moitié l'entrée dans les deux dimensions spatiales.</li>
    <li> Strides,  Padding et data_format ont la même signification que dans la couche de convolution. S'ils ne sont pas précisés dans la fonction de MaxPooling2D, ils prendont les mêmes valeurs que dans la couche de convolution. </li>
</p>

**La couche de correction :** 

La couche de correction ReLU remplace toutes les valeurs négatives reçues en entrées par des zéros. Elle joue le rôle de fonction d'activation. Elle est appliquée dans la couche de convolution de notre modèle :  activation = 'relu'. La fonction 'relu' est très utilisée dans les CNN. Elle permet un apprentissage plus rapide de notre modéle.

Ces deux commandes sont les mêmes :

 - model.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3))
 
 - model.add(layers.Conv2D(32, (3, 3), input_shape = (150, 150, 3))       
 model.add(layers.Activation(activations.relu)) </li>


**La couche fully-connected**

La couche fully-connected constitue toujours la dernière couche d'un réseau de neurones, convolutif ou non – elle n'est donc pas caractéristique d'un CNN. 
Ce type de couche reçoit un vecteur en entrée et produit un nouveau vecteur en sortie. Pour cela, elle applique une combinaison linéaire puis éventuellement une fonction d'activation aux valeurs reçues en entrée.
La dernière couche fully-connected permet de classifier l'image en entrée du réseau : ici elle renvoie un vecteur de taille 2 ( pour une classification binaire). Chaque élément du vecteur indique la probabilité pour l'image en entrée d'appartenir à une classe (malin ou bénin). 

La fonction flatten : Elle permet de convertir des matrices 3D en vecteur 1D.

La fonciton Dense :     
*tf.keras.layers.Dense( units, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)*
<p>
    <li> Units :Entier positif précisant la taille du vecteur en sortie</li>
    <li> Activation : indique si une correction ReLU ou softmax est effectuée juste après la couche fully-connected </li>
    <li> Les autres paramétres ont la même description que dans la couche de convolution </li>
</p>

**Compilation du modéle :**

Avant d’entraîner notre modèle, nous devons configurer le processus d’apprentissage en appelant la méthode compile() de Keras.

*compile( optimizer, loss = None, metrics = None, loss_weights = None, sample_weight_mode = None, weighted_metrics = None, target_tensors = None )*

<p>
    <li>optimizer : optimise les poids d'entrée en comparant la fonction de prédiction et de perte. Keras fournit plusieurs optimiseurs sous forme de module. Pour un probleme de classification binaire on utilisera 'rmsprop'</li>

<li>loss : Il s’agit de la fonction de coût que le modèle va utiliser pour minimiser les erreurs. On utilisera loss='binary_crossentropy' pour une classification binaire,</li>

<li> metrics : En machine learning, les métriques sont utilisées pour évaluer les performances du modèle. Il est similaire à la fonction de perte, mais n'est pas utilisé dans le processus de formation. Keras fournit plusieurs métriques sous forme de module. Dans le cas d'un probleme de classification on utilise metrics = ['accuracy']</li>

<li> loss_weights, sample_weight_mode, weighted_metrics, target_tensors : Parametres de pondération facultatifs. Nous garderons ces parametres à leur valeur par défaut.   </li>

##### c) training

In [ ]:
history = model2.fit(train_images, train_labels, batch_size=128, epochs=20, validation_split = 0.2)

In [ ]:
test_loss = model2.evaluate(test_images, test_labels)

Visualisation de la courbe accuracy au cours des époques :

In [ ]:
plt.plot(history.history['accuracy'], color='b', label="Training accuracy")
plt.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
plt.title('Training and validation accuracy')

legend = plt.legend()
plt.tight_layout()
plt.show()

Visualisation de la courbe loss au cours des époques :

In [ ]:
plt.plot(history.history['loss'], color='b', label="Training loss")
plt.plot(history.history['val_loss'], color='r', label="validation loss")
plt.title('Training and validation loss')

legend = plt.legend()
plt.show()

Représentation de la matrice de confusion :

In [ ]:
predictions = model2.predict(test_images)     
pred_labels = np.argmax(predictions, axis = 1) 

In [ ]:
CM = sklearn.metrics.confusion_matrix(test_labels, pred_labels)
ax = plt.axes()
sns.heatmap(CM, annot=True, 
           annot_kws={"size": 10}, 
           xticklabels=class_names, 
           yticklabels=class_names, ax = ax)
ax.set_title('Confusion matrix')
plt.show()

#### 2 - Améliorations du modéle

Nous souhaitons améliorer la performance de notre modèle. Pour celà, nous pouvons modifier plusieurs aspects de notre réseau de neurones : 
- Les couches : nous pouvons en ajouter ou en supprimer. Nos images sont assez complexes (dimension 150 x 150, images de couleurs... ), c'est pourquoi nous avons besoin de plusieurs couches.
- Ajouter des fonctions : 
    - Dropout(p) : Cette fonction permet de limiter le sur-apprentissage en abandonnant avec probabilité p certains neurones. 
    - La fonction BatchNormalization() : elle est utilisée dans de nombreux réseaux de neurones afin d'élever le taux d'apprentissage des modèles en réduisant la sensibilité des réseaux de neurones aux poids de départ. Malgrè plusieurs tentatives, cette fonction n'a pas permis l'amélioration de notre modèle, elle ne sera donc pas ajouter dans la suite de ce rapport.   
- Les hypers paramétres : Beaucoup d'hypers paramétres sont présents dans la création du modéle ; le nombre de filtres, les fonctions d'activation, la taille du vecteur de sortie de la fonction Dense (units), le paramétre p de la fonction Dropout(p), le learning rate... Ces hypers paramétres sont modifiables. Nous allons créer une fonction permettant de tester différents hypers paramétres afin de récupérer les paramétres rendant notre modéle optimal. 
 

In [25]:
# IIIIIIIIIIIIIIIIIICCCCCCCCCCCCCCIIIIIIIIIIIIIIIIIIIIIII
# Importation des packages :

from tensorflow import keras
from tensorflow.keras.layers import (Conv2D, Dense,Dropout, Flatten,MaxPooling2D)

from hyperopt import hp # pour une première utilisation : pip install hyperopt
import kerastuner as kt # pour une première utilisation :  pip install keras-tuner
hp = kt.HyperParameters()

from kerastuner import HyperModel

from kerastuner.tuners import RandomSearch

from kerastuner.tuners import Hyperband

In [26]:
# IIIIIIIIIIICCCCCCCCCCCCCCCCCCCCCCCIIIIIIIIIIIIIIIIIIII et toutes les cellules jusqu'à la fonction des enfers 
# On indique les hypers parametres que nous voulons tester ainsi que les choix possibles : 

# Le nombre de filtres 
filters=hp.Choice('num_filters', [16,32,64,128,256])

# La taille du vecteur de sortie et la fonction d'activation 
Dense(units=hp.Int('units', 32, 512, step=32), activation=hp.Choice('dense_activation',['relu','tanh','sigmoid']))

# Le learning rate 
hp.Float('learning_rate',min_value=1e-5, max_value=1e-2, sampling='LOG',default=1e-3)

0.001

In [27]:
from kerastuner import HyperModel

INPUT_SHAPE = (150,150,3)
NUM_CLASSES = 2

# Création de la classe CNNHyperModel permettant de modifier les parametres souhaités

class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.Sequential()
        model.add( Conv2D( filters=16, kernel_size=5,activation="relu",input_shape=self.input_shape,))
        model.add(Conv2D(filters=hp.Choice("num_filters_1", values=[32,16], default=32,),activation="relu",kernel_size=3,))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Dropout(rate=hp.Float("dropout_1", min_value=0.0, max_value=0.5, default=0.25, step=0.05,)))
        
        model.add(Conv2D(filters=32, kernel_size=3, activation="relu"))
        model.add(Conv2D(filters=hp.Choice("num_filters_2", values=[32, 64], default=64,),activation="relu",kernel_size=3,))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Dropout(rate=hp.Float("dropout_2", min_value=0.0, max_value=0.5, default=0.25, step=0.05,)))
        
        model.add(Conv2D(filters=64, kernel_size=3, activation="relu"))
        model.add(Conv2D(filters=hp.Choice("num_filters_3", values=[64,128], default=128,),activation="relu",kernel_size=3,))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Dropout(rate=hp.Float("dropout_3", min_value=0.0, max_value=0.5, default=0.25, step=0.05,)))
        
        model.add(Conv2D(filters=128, kernel_size=3, activation="relu"))
        model.add(Conv2D(filters=hp.Choice("num_filters_4", values=[128,256], default=256,),activation="relu",kernel_size=3,))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Dropout(rate=hp.Float("dropout_4", min_value=0.0, max_value=0.5, default=0.25, step=0.05,)))
        
        model.add(Flatten())
        model.add(Dense(units=hp.Int("units", min_value=32, max_value=512, step=32, default=128),activation=hp.Choice('dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu',),))
        model.add(Dropout(rate=hp.Float("dropout_5", min_value=0.0, max_value=0.5, default=0.25, step=0.05 )))
        model.add(Dense(self.num_classes, activation="softmax"))

        model.compile(optimizer=keras.optimizers.Adam(hp.Float("learning_rate",min_value=1e-4,max_value=1e-2,sampling="LOG",default=1e-3,)
            ),loss="sparse_categorical_crossentropy",metrics=["accuracy"],)
        return model
    
    
hypermodel = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)
                 
    

In [28]:
# AJOUTER DESCRIPTION
HYPERBAND_MAX_EPOCHS = 40
MAX_TRIALS = 20
EXECUTION_PER_TRIAL = 2

In [29]:
SEED= 1
from kerastuner.tuners import RandomSearch
hypermodel=CNNHyperModel (input_shape = INPUT_SHAPE, num_classes=NUM_CLASSES)

tuner = RandomSearch(hypermodel, objective='val_accuracy', seed = SEED, max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL, directory = 'random_search', project_name='CNN_melanome')

INFO:tensorflow:Reloading Oracle from existing project random_search\CNN_melanome\oracle.json


In [30]:
from kerastuner.tuners import Hyperband

tuner = Hyperband(hypermodel, max_epochs= HYPERBAND_MAX_EPOCHS, objective = 'val_accuracy', seed = SEED, executions_per_trial=EXECUTION_PER_TRIAL, directory='hyperband', project_name='CNN_melanome')

INFO:tensorflow:Reloading Oracle from existing project hyperband\CNN_melanome\oracle.json


In [31]:
tuner.search_space_summary()

Search space summary
Default search space size: 12
num_filters_1 (Choice)
{'default': 32, 'conditions': [], 'values': [32, 16], 'ordered': True}
dropout_1 (Float)
{'default': 0.25, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.05, 'sampling': None}
num_filters_2 (Choice)
{'default': 64, 'conditions': [], 'values': [32, 64], 'ordered': True}
dropout_2 (Float)
{'default': 0.25, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.05, 'sampling': None}
num_filters_3 (Choice)
{'default': 128, 'conditions': [], 'values': [64, 128], 'ordered': True}
dropout_3 (Float)
{'default': 0.25, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.05, 'sampling': None}
num_filters_4 (Choice)
{'default': 256, 'conditions': [], 'values': [128, 256], 'ordered': True}
dropout_4 (Float)
{'default': 0.25, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.05, 'sampling': None}
units (Int)
{'default': 128, 'conditions': [], 'min_value': 32, 'max_value': 512

In [34]:
# Attention cette cellule peut mettre plusieurs heures à aboutir. 
# En effet, l'algorithme va tester toutes les combinaisons possibles pour les hypers parametres 
N_EPOCH_SEARCH = 40
tuner.search(train_images, train_labels, epochs = N_EPOCH_SEARCH, validation_split=0.1)


Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
num_filters_1     |32                |?                 
dropout_1         |0.5               |?                 
num_filters_2     |32                |?                 
dropout_2         |0.1               |?                 
num_filters_3     |128               |?                 
dropout_3         |0.4               |?                 
num_filters_4     |256               |?                 
dropout_4         |0.1               |?                 
units             |256               |?                 
dense_activation  |sigmoid           |?                 
dropout_5         |0.25              |?                 
learning_rate     |0.00088949        |?                 
tuner/epochs      |2                 |?                 
tuner/initial_e...|0                 |?                 
tuner/bracket     |3                 |?                 
tuner/round       |0                 |?                 

Epo

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 250: invalid start byte

In [36]:
train_labels

array([1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0.,
       1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
       0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0.,
       1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1.,
       1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1.,
       0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0.,
       1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
       1., 0., 1., 0., 1.

In [ ]:
# Show a summary of the search
tuner.results_summary()

# Retrieve the best model.
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model.
loss, accuracy = best_model.evaluate(test_images, test_labels)

bon là il y a encore plein de trucs
- à améliorer (changer les différents paramètres jusqu'à trouver les meilleurs)
- à ajouter : des représentations de la loss et de l'accuracy au cours des epochs, un pré-traitement des images, une liste des images avec pour chacun la probabilité de malignité, une fonction où on rentrer une unique image et il dit si elle est maligne ou bénine... 
Je pense qu'on peut vraiment faire des trucs funs !

### 5 - Modèle pré-entraîné

Enfin, on essaie d'améliorer la fiabilité de notre prédiction en utilisant un modèle pré-entraîné, à travers le principe du **transfer learning**. Le principe est d'utiliser les connaissances acquises par un réseau de neurones entraîné sur un grand nombre d'images, et d'appliquer ces connaissances à notre problème particulier (la reconnaissance de mélanomes). 

Pour ce faire, nous utiliserons **VGG16** : il s'agit d'un réseau de neurones construit par K. Simonyan et A. Zisserman (univeristé d'Oxford), considéré comme l'un des meilleurs dans le champs de la vision. Comme son nom l'indique, ce CNN est constitué de 16 couches. 

De plus, nous utiliserons des poids pré-établis pour ce CNN sur le dataset **ImageNet** (un jeu de données de plus de 14 millions d'images appartenant à 1 000 classes différentes).


In [ ]:
# import des modules 

from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

VGG16 prend en entrée des images de taille 224x224, il nous faut donc re-charger nos images (précédemment en taille 150x150) à la bonne taille :

In [ ]:
image_size_VGG = 224

In [ ]:
path_jpg_train = Path_Projet_Melanomes+'/JPG_Sample_Train_Resize' 
path_jpg_test = Path_Projet_Melanomes+'/JPG_Sample_Test_Resize' 
    
meta_data = df_sample
meta_data = meta_data.set_index("image_id")
datasets = [path_jpg_train, path_jpg_test]

output_VGG = []

#itération sur chaque dataset (train puis test)
for dataset in datasets :

    files = os.listdir(dataset)
    images = np.zeros((len(files), image_size_VGG, image_size_VGG, 3), dtype=np.float32)
    labels = np.zeros(len(files))

    # itération sur chaque image du dataset
    for i, file in tqdm(enumerate(files)):
        
        # obtention du le chemin de l'image
        img_path = os.path.join(dataset, file)

        # obtention du label de l'image
        label = meta_data.loc[file[0:12]].target

        # ouverture de l'image et retraitement de l'image (changement de couleurs et réduction de taille)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (image_size_VGG, image_size_VGG))

        # ajout de l'image et de son label à la liste de résultats
        images[i, :] = (np.array(image) / 255).astype(np.float32)
        labels[i] = label

        gc.collect()

    output_VGG.append((images, labels))

In [ ]:
(train_images_VGG, train_labels_VGG), (test_images_VGG, test_labels_VGG) = output_VGG

In [ ]:
train_images_VGG, train_labels_VGG = sklearn.utils.shuffle(train_images_VGG, train_labels_VGG, random_state=25)

In [ ]:
train_images

On construit ensuite le réseau de neurones, à partir de VGG16 et des poids ImageNet

In [ ]:
# consutruction du CNN

# les couches de base du CNN sont celles du modèle pré-entraîné, avec les poids issus de ce pré-entraînement (imagenet)
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(image_size_VGG, image_size_VGG, 3))

# on ajoute ensuite quelques couches correspondant plus particulièrement à notre problème 
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(2, activation='sigmoid'))

# on assemble les deux 
model = Model(inputs=base_model.input, outputs=add_model(base_model.output))

On compile le modèle 

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
#model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),metrics=['accuracy'])

In [ ]:
train_labels_VGG = np.asarray(train_labels_VGG).astype('float32').reshape((-1,1))
test_labels_VGG = np.asarray(test_labels_VGG).astype('float32').reshape((-1,1))

In [ ]:
history = model.fit(train_images_VGG, train_labels_VGG, batch_size=32, epochs=20, validation_split = 0.2)

In [ ]:
batch_size = 32
epochs = 10 #à augmenter quand on aura le temps (ça prend 3h à tourner)

In [ ]:
# preprocessing des images
train_datagen = ImageDataGenerator(
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True)
train_datagen.fit(train_images_VGG)

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
train_labels_VGG = np.asarray(train_labels_VGG).astype('float32').reshape((-1,1))
test_labels_VGG = np.asarray(test_labels_VGG).astype('float32').reshape((-1,1))

In [ ]:
history = model.fit(train_images_VGG, train_labels_VGG, batch_size=batch_size, steps_per_epoch=train_images_VGG.shape[0] // batch_size,
    epochs=epochs,
    validation_data=(test_images_VGG, test_labels_VGG),
    callbacks=[ModelCheckpoint('VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)])

In [ ]:
hist_as_df = pd.DataFrame(history.history)
hist_as_df

In [ ]:
predictions = model.predict(test_images_VGG)     # Vector of probabilities
pred_labels = np.argmax(predictions, axis = 1) # We take the highest probability

In [ ]:
CM = sklearn.metrics.confusion_matrix(pred_labels, test_labels_VGG)
ax = plt.axes()
sns.heatmap(CM, annot=True, 
           annot_kws={"size": 10}, 
           xticklabels=class_names, 
           yticklabels=class_names, ax = ax)
ax.set_title('Confusion matrix')
plt.show()